# ExKF

## 実装の設定
- モデルの遷移関数$M$は非線形なのでforecast stepの予報誤差共分散行列を計算する際に接線形近似を行う必要がある．
  - サイクルのステップ幅を接線形近似が成り立つ範囲でとる必要がある．
- 最初は完全モデルを仮定($Q=0$)
- 観測$H$は単位行列$I$もしくはそのrankを落とした単位行列を使う．
- $R = I $で観測データを生成したのでこれを使う．のちに$ R = rI$として$r$を変化させる．
- 初期値:
    - $x_0$: attractor上のランダムな点をとる
    - $P_0 = 25I$: attractorの平均距離の2乗. 大きく取れば問題ない．


In [ ]:
%matplotlib inline
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

# モジュールの読み込み
import sys
sys.path.append('./')
from utils import make_lorenz96, rk4, error_series_kf, plot_error_KF, estimate_error_with_params

# ExKFをimport
from kalman_filters import ExtendedKalmanFilter as ExKF

In [ ]:
# Lorenz96の設定
J = 40
F = 8
lorenz = make_lorenz96(F)

# 同化step
# 時間発展は0.01ごとに行う
dt = 0.05

# モデルの遷移関数(非線形)
# 0.01ずつ時間発展させる
# dtは同化step
def M(x, dt):
    for i in range(int(dt/0.01)):
        x = rk4(0, x, 0.01, lorenz)
    return x

# 単位行列
I = np.identity(J)

# 観測
H = I

# モデル誤差共分散, 最初は完全モデルを仮定
Q = np.zeros((J, J))

# 観測誤差共分散, 後で定数倍の変化をさせる.
R = I

# 観測値と真値
end_step = 500 # 開発用
y = np.load('data/obs_atr.npy')
true = np.load('data/true_atr.npy')

# KFの初期値
np.random.seed(1)
x_0 = true[np.random.randint(len(true)-1)]
P_0 = 25*I

## demo

In [ ]:
%%time
# 実行済
# exkf = ExKF(M, H, Q, R, y, x_0, P_0, dim_x=J, dt=dt)
# exkf.forward_estimation()
# np.save('data/exkf/exkf.npy', exkf.x)

In [ ]:
exkf_x = np.load('data/exkf/exkf.npy')
plot_error_KF(true, y, [exkf_x])

In [ ]:
観察
- 最初は推定誤差が観測誤差より小さくなっているが，ある時刻を超えると観測誤差を超えて大きくなってしまっている．
  - 原因: 予報誤差共分散が時間経過とともに小さくなってしまって，モデルを信用しすぎてデータに含まれるノイズについていけていない．
  - 対策: 誤差共分散膨張により予報誤差共分散が小さくなりすぎるのを防ぐ．

## ExKFの観測分布への依存性を調べる

*間違いがある*

間引き方
- 空間方向; 観測行列$H$のrankを落とした際の挙動を調べる．
    - 端の観測値から順に間引く． -> 海陸分布
    - 等間隔で間引く. -> gridが荒い
    - 各時刻でランダムに間引く
- 時間方向
    - 同化ステップ幅を倍の0.1(12h)にする．

#### 端の観測値から順に間引く

In [ ]:
# 実行済
# H_dege = I.copy()
# obs_dege_results = []
# for j in range(5):
#     H_dege[j, j] = 0
#     H_dege[J-1-j, J-1-j] = 0
#     exkf_obs_dege = ExKF(M, H=H_dege, Q, R, y, x_0, P_0, dt=dt)
#     exkf_obs_dege.forward_estimation()
#     obs_dege_results.append(exkf_obs_dege.x)
# np.save('data/exkf/exkf_obs_dege_array.npy', np.array(obs_dege_results))

In [ ]:
exkf_obs_dege_results = np.load('data/exkf/exkf_obs_dege_array.npy')
legends = ['rank H = 38', 'rank H = 36', 'rank H = 34', 'rank H = 32', 'rank H = 30']
plot_error_KF(true, y, exkf_obs_dege_results, legends)

inflationあり

In [ ]:
# 旧s
# H_dege = I.copy()
# obs_dege_results = []
# for j in range(20):
#     H_dege[2*j, 2*j] = 0
#     exkf_obs_dege = ExKF(M, H=H_dege, Q, R, y, x_0, P_0, dt=dt, alpha=1.1)
#     exkf_obs_dege.forward_estimation()
#     obs_dege_results.append(exkf_obs_dege.x)
# np.save('data/exkf/exkf_obs_dege_array_with_inflation.npy', np.array(obs_dege_results))

In [ ]:
# H_dege = I.copy()
# obs_dege_results = []
# for j in range(20):
#     H_dege[2*j, 2*j] = 0
# exkf_obs_dege = ExKF(M, H_dege, Q, R, y, x_0, P_0, dt=dt, alpha=1.22)
# exkf_obs_dege.forward_estimation()
# obs_dege_results.append(exkf_obs_dege.x)
# np.save('data/exkf/exkf_obs_20dege_array_with_inflation.npy', np.array(obs_dege_results))

In [ ]:
exkf_obs_dege_results_with_inf = np.load('data/exkf/exkf_obs_20dege_array_with_inflation.npy')
plot_error_KF(true[100:], y[100:], [exkf_obs_dege_results_with_inf[0][100:]])

### 20個まで観測を抜く(偶数番目)
- 各$ H $で$\alpha = 1.0 \sim 1.3$で探索
- 推定1h40min -> 結果: 1h36min


In [ ]:
# %%time
# H_dege = I.copy()
# optimal_alpha_by_H = []
# for j in range(20):
#     H_dege[2*j, 2*j] = 0
#     obs_dege_results = []
#     params_alpha = []
#     for k in range(6):
#         inf = 0.05*k
#         params_alpha.append(1+inf)
#         exkf_obs_dege = ExKF(M, H_dege, Q, R, y, x_0, P_0, dt=dt, alpha=1+inf)
#         exkf_obs_dege.forward_estimation()
#         obs_dege_results.append(exkf_obs_dege.x)
#     optimal_alpha, optimal_idx = estimate_error_with_params(true, obs_dege_results, params_alpha, 'alpha', plot=False)
#     optimal_alpha_by_H.append(optimal_alpha)
#     optimal_result = obs_dege_results[optimal_idx]
#     np.save('data/exkf/exkf_obs_{}dege_inflation_parametrized.npy'.format(j), np.array(obs_dege_results))
#     np.save('data/exkf/exkf_obs_{}dege_optimal.npy'.format(j), optimal_result)
# np.save('data/exkf/exkf_optimal_alpha_for_obs_0to20dege.npy', np.array(optimal_alpha_by_H))

In [ ]:
# 失敗したので再度実行
# %%time
# optimal_alpha_by_H = np.load('data/exkf/exkf_optimal_alpha_for_obs_0to20dege.npy')
# H_dege = I.copy()
# for j in range(20):
#     H_dege[2*j, 2*j] = 0
#     exkf_obs_dege = ExKF(M, H_dege, Q, R, y, x_0, P_0, dt=dt, alpha=optimal_alpha_by_H[j])
#     exkf_obs_dege.forward_estimation()
#     np.save('data/exkf/exkf_obs_{}dege_optimal.npy'.format(j), exkf_obs_dege.x)

In [ ]:
params_H = range(1,21)
results_by_H = [np.load('data/exkf/exkf_obs_{}dege_optimal.npy'.format(j-1)) for j in params_H]
_ = estimate_error_with_params(true, results_by_H, params_H, 'alpha', plot=True)

#＃# 観察
- 観測を減らすごとに右肩上がりになっているがdivergenceは起きていない．

## 各時刻でランダムに間引く

パラメータ
- 共分散膨張ありなし
- cut size 20くらい

- 各cut sizeで平均をとる． サンプル数5くらい


In [ ]:
# %%time
# results = []
# max_cut_size = 20
# n_samples = 5
# for cut_obs_size in range(max_cut_size):
#     print('===========\n')
#     print('cut_obs_size: {}'.format(cut_obs_size))
    
#     exkf_random_cut_obs_samples = []
#     for n in range(n_samples):
#         print('---------------')
#         print('n: {}'.format(n))
        
#         exkf_random_cut_obs = ExKF(M, H, Q, R, y, x_0, P_0, dim_x=J, dt=dt, alpha=1.1, cut_obs_size=cut_obs_size)
#         exkf_random_cut_obs.forward_estimation()
#         exkf_random_cut_obs_samples.append(exkf_random_cut_obs.x)
#     results.append(np.array(exkf_random_cut_obs_samples).mean(axis=0))
# np.save('data/exkf/exkf_random_cut_obs_{}parametrized_{}samples.npy'.format(max_cut_size, n_samples), results)

In [ ]:
max_cut_size = 20
n_samples = 5
exkf_random_cut_obs_results = np.load('data/exkf/exkf_random_cut_obs_{}parametrized_{}samples.npy'.format(max_cut_size, n_samples))
params_cut_obs = range(max_cut_size)
legends = ['cut size = {}'.format(n) for n in params_cut_obs]
plot_error_KF(true, y, exkf_random_cut_obs_results, legends)
estimate_error_with_params(true, exkf_random_cut_obs_results, params_cut_obs, 'cut size ({}sample mean)'.format(n_samples), log=False)

- 綺麗な右肩上がりにはならなかった.
- サンプル数が少なかったのかも.
- 1を超えてrmseが悪くなっているものはない.


### 20~39
- 誤差共分散膨張1.1
- cut size: 20 ~ 39
- 各cut sizeで5サンプルの平均をとる

In [ ]:
# %%time
# results = []
# max_cut_size = 40
# n_samples = 5
# for cut_obs_size in range(20, max_cut_size):
#     print('===========\n')
#     print('cut_obs_size: {}'.format(cut_obs_size))
    
#     exkf_random_cut_obs_samples = []
#     for n in range(n_samples):
#         print('---------------')
#         print('n: {}'.format(n))
        
#         exkf_random_cut_obs = ExKF(M, H, Q, R, y, x_0, P_0, dim_x=J, dt=dt, alpha=1.1, cut_obs_size=cut_obs_size)
#         exkf_random_cut_obs.forward_estimation()
#         exkf_random_cut_obs_samples.append(exkf_random_cut_obs.x)
#     results.append(np.array(exkf_random_cut_obs_samples).mean(axis=0))
# np.save('data/exkf/exkf_random_cut_obs_{}parametrized_{}samples.npy'.format(max_cut_size, n_samples), results)

In [ ]:
max_cut_size = 40
n_samples = 5
exkf_random_cut_obs_results = np.load('data/exkf/exkf_random_cut_obs_{}parametrized_{}samples.npy'.format(max_cut_size, n_samples))
params_cut_obs = range(20, 28)
legends = ['cut size = {}'.format(n) for n in params_cut_obs]
plot_error_KF(true, y, exkf_random_cut_obs_results[:8], legends)
estimate_error_with_params(true, exkf_random_cut_obs_results, params_cut_obs, 'cut size ({}sample mean)'.format(n_samples), log=False)

### 観察
- 実行時間: 9298.583082914352 [sec]
- rmseが1を超えるものも出てくる
- 28個除いた時点で計算が発散している．

## ExKFのパラメータ依存性
1. $R = rI$として$r$を変化させる． ($r=1$で観測データを生成しているので$r=1$が正しい．）
2. $Q = qI$として$q$を変化させる．

仮説
- $r$を大きくするとモデルに近づく.
- $q$を大きくするとデータに近づく.


### 観測誤差$R$を変化させる
- $r$の値を指数的に増加させる．

In [ ]:
# 実行済
# results_exkf_r = []
# params_r = []
# for k in range(7):
#     r = 10**(k-3)
#     params_r.append(r)
#     R_r = r*I
#     exkf_r = ExKF(M, H, Q, R_r, y, x_0, P_0, dim_x=J, dt=dt)
#     exkf_r.forward_estimation()
#     results_exkf_r.append(exkf_r.x)
# np.save('data/exkf/exkf_r_parametrized.npy', np.array(results_exkf_r))

In [ ]:
exkf_results_r = np.load('data/exkf/exkf_r_parametrized.npy')
params_r = [10**(k-3) for k in range(7)]
legends_r = ['r = {}'.format(r) for r in params_r]
plot_error_KF(true, y, exkf_results_r, legends = legends_r)
estimate_error_with_params(true, exkf_results_r, params_r, 'r', log=True)

### 観察
- $r$を大きくすると途中で破綻しやすくなっている．
  - 原因: 相対的にモデルの信頼度が上がるため．

### モデル誤差$Q$を変化させる
モデル誤差を導入．しかし真の時間並進作用素を知っているので不適切かも．
TODO: $M$を近似したモデルを作る． -> [課題9](#課題9)
- $q$の値を指数的に増加させる．

In [ ]:
# 実行済
# results_exkf_q = []
# params_q = []
# for k in range(7):
#     q = 10**(k-3)
#     params_q.append(q)
#     Q_q = q*I
#     var3d_q = ExKF(M, H, Q_q, R, y, x_0, P_0, dim_x=J, dt=dt)
#     var3d_q.forward_estimation()
#     results_exkf_q.append(var3d_q.x)
# np.save('data/exkf/exkf_q_parametrized.npy', np.array(results_exkf_q))

In [ ]:
exkf_result_q = np.load('data/exkf/exkf_q_parametrized.npy')
params_q = [10**(k-3) for k in range(7)]
legends_q = ['q = {}'.format(q) for q in params_q]
plot_error_KF(true, y, exkf_result_q, legends=legends_q)
estimate_error_with_params(true, exkf_result_q, params_q, 'q', log=True)

### 観察
- 途中で破綻せずに推定を行うことができている．
- $q$を大きくすると推定誤差が観測誤差に近づく．

## 誤差共分散膨張
他のパラーメータが変わった場合は最適値が変わる
- additive inflation: $ P^f = P^f + \alpha I $と置き換える． 足す行列を吟味する必要がある．
- multiplicative inflation: $ P^f = \alpha P^f $
- relaxation to prior: 元の値に近づける． ExKFではあまりやらないかも．
- relaxation to prior spread: 

### 加法的誤差共分散膨張
$ P^f = P^f + \alpha I $

In [ ]:
# ===========================
# 注意: 旧verの実装なのでこのまま実行するとうまくいかない
# ===========================
# 実行済 加法的誤差共分散膨張のver
# results_exkf_inf = []
# params_inf = []
# for k in range(5):
#     inf = 10**(-k)
#     params_inf.append(inf)
#     exkf_inf = ExKF(M, H, Q, R, y, x_0, P_0, dim_x=J, dt=dt, alpha=inf)
#     exkf_inf.forward_estimation()
#     results_exkf_inf.append(exkf_inf.x)
# np.save('data/exkf/exkf_inflation_parametrized.npy', np.array(results_exkf_inf))

In [ ]:
params_inf = [10**(-k) for k in range(5)]
exkf_inf_results = np.load('data/exkf/exkf_inflation_parametrized.npy')
legends_inf = ['alpha = {}'.format(a) for a in params_inf]
plot_error_KF(true, y, exkf_inf_results, legends=legends_inf)
estimate_error_with_params(true, exkf_inf_results, params_inf, 'inflation factor alpha', log=True)
estimate_error_with_params(true, exkf_inf_results[3:], params_inf[3:], 'additive inflation factor alpha', log=True)

### 乗法的誤差共分散膨張
<!--$P^f = \alpha P^f $ -->

$ P^f = (1 + \alpha) P^f $の$ \alpha$を変える

In [ ]:
# 実行済
# results_exkf_multi_inf = []
# for k in range(7):
#     inf = 10**(k-3)
#     exkf_multi_inf = ExKF(M, H, Q, R, y, x_0, P_0, dim_x=J, dt=dt, alpha=1 + inf)
#     exkf_multi_inf.forward_estimation()
#     results_exkf_multi_inf.append(exkf_multi_inf.x)
# np.save('data/exkf/exkf_multi_inflation_parametrized.npy', np.array(results_exkf_multi_inf))

In [ ]:
params_multi_inf = [10**(k-3) for k in range(7)]
exkf_multi_inf_results = np.load('data/exkf/exkf_multi_inflation_parametrized.npy')
legends_multi_inf = ['alpha = {}'.format(a) for a in params_multi_inf]
plot_error_KF(true, y, exkf_multi_inf_results, legends=legends_multi_inf)
estimate_error_with_params(true, exkf_multi_inf_results, params_multi_inf, 'inflation factor alpha', log=True)
estimate_error_with_params(true, exkf_multi_inf_results[2:4], params_multi_inf[2:4], 'inflation factor alpha', log=True)

### 観察
- 加法的な場合は$ + 10^{-4} I $が良さそう．
- 乗法的な場合は1.1倍くらいが良さそう．
- 加法的な方がrmseの時間平均が小さくなっている．

#### 乗法的細かく探索
$ \alpha $ を$ 0.01 \sim 0.1$で探索

In [ ]:
# results_exkf_multi_inf_detail = []
# for k in range(30):
#     inf = 0.08 + 0.0002*k
#     exkf_multi_inf_detail = ExKF(M, H, Q, R, y, x_0, P_0, dim_x=J, dt=dt, alpha=1 + inf)
#     exkf_multi_inf_detail.forward_estimation()
#     results_exkf_multi_inf_detail.append(exkf_multi_inf_detail.x)
# np.save('data/exkf/exkf_multi_inflation_detail_parametrized.npy', np.array(results_exkf_multi_inf_detail))

In [ ]:
params_multi_inf_detail = [0.08 + 0.0002*k for k in range(30)]
exkf_multi_inf_detail_results = np.load('data/exkf/exkf_multi_inflation_detail_parametrized.npy')
legends_multi_inf_detail = ['alpha = {}'.format(a) for a in params_multi_inf_detail]
plot_error_KF(true, y, exkf_multi_inf_detail_results, legends=legends_multi_inf_detail)
estimate_error_with_params(true, exkf_multi_inf_detail_results, params_multi_inf_detail, 'inflation factor alpha', log=False)

### 観察
- $0.0842$が最適．
- これ以降inflation時に$1.0842$倍をする．


## 最適なExKFの結果を保存


In [ ]:
# %%time
# exkf_opti = ExKF(M, H, Q, R, y, x_0, P_0, dt=dt, alpha=1.0842)
# exkf_opti.forward_estimation()
# np.save('data/exkf/exkf_opti.npy', exkf_opti.x)

## memo

In [ ]:
# 誤差共分散膨張の失敗時の記録
# end = 320
# p_hist_q = np.load('data/p_hist_q.npy')
# p_hist_inflation = np.load('data/p_hist_inflation.npy')
# end = 320
# plt.plot(np.trace(p_hist_q[1:end]), label='Q')
# plt.plot(np.trace(p_hist_inflation[1:end]), label='inflation')
# plt.legend()